In [32]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


import tensorflow as tf
from tensorflow.keras.optimizers import SGD
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

In [14]:
df = pd.read_csv(r"C:\Users\trush\OneDrive\Documents\WFU Grad School Info\BAN 6025 Machine Learning\Sky's the Limit\BAN6025Project2Data.csv")
df.head()

,Target_Y,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31
0,0,0.3,0.986506,-1,25,40,0.006735,102.453711,AA,1059,...,0,1500,0,INTERNET,16.224843,linux,1,1,0,0
1,0,0.8,0.617426,-1,89,20,0.010095,-0.849551,AD,1658,...,0,1500,0,INTERNET,3.363854,other,1,1,0,0
2,0,0.8,0.996707,9,14,40,0.012316,-1.490386,AB,1095,...,0,200,0,INTERNET,22.730559,windows,0,1,0,0
3,0,0.6,0.475100,11,14,30,0.006991,-1.863101,AB,3483,...,0,200,0,INTERNET,15.215816,linux,1,1,0,0
4,0,0.9,0.842307,-1,29,40,5.742626,47.152498,AA,2339,...,0,200,0,INTERNET,3.743048,other,0,1,0,0


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 32 columns):
 #   Column    Non-Null Count    Dtype  
---  ------    --------------    -----  
 0   Target_Y  1000000 non-null  int64  
 1   X1        1000000 non-null  float64
 2   X2        1000000 non-null  float64
 3   X3        1000000 non-null  int64  
 4   X4        1000000 non-null  int64  
 5   X5        1000000 non-null  int64  
 6   X6        1000000 non-null  float64
 7   X7        1000000 non-null  float64
 8   X8        1000000 non-null  object 
 9   X9        1000000 non-null  int64  
 10  X10       1000000 non-null  float64
 11  X11       1000000 non-null  float64
 12  X12       1000000 non-null  float64
 13  X13       1000000 non-null  int64  
 14  X14       1000000 non-null  int64  
 15  X15       1000000 non-null  object 
 16  X16       1000000 non-null  int64  
 17  X17       1000000 non-null  int64  
 18  X18       1000000 non-null  object 
 19  X19       1000000 non-

In [16]:
df = pd.get_dummies(df, columns=['X8', 'X18', 'X15', 'X25', 'X27'], drop_first=True, dtype=float)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 48 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   Target_Y       1000000 non-null  int64  
 1   X1             1000000 non-null  float64
 2   X2             1000000 non-null  float64
 3   X3             1000000 non-null  int64  
 4   X4             1000000 non-null  int64  
 5   X5             1000000 non-null  int64  
 6   X6             1000000 non-null  float64
 7   X7             1000000 non-null  float64
 8   X9             1000000 non-null  int64  
 9   X10            1000000 non-null  float64
 10  X11            1000000 non-null  float64
 11  X12            1000000 non-null  float64
 12  X13            1000000 non-null  int64  
 13  X14            1000000 non-null  int64  
 14  X16            1000000 non-null  int64  
 15  X17            1000000 non-null  int64  
 16  X19            1000000 non-null  int64  
 17  X20      

In [ ]:
def stepwise_selection(X, y,
                       initial_list=[],
                       threshold_in=0.01,
                       threshold_out = 0.05,
                       verbose=True):
    """ Perform a forward-backward feature selection
    based on p-value from statsmodels.api.OLS

    Arguments:
        X - pandas.DataFrame of numeric features
        y - vector, series of the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions

    Returns: list of selected features

    Example Call: stepwise_selection(X, y)
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print(f'Add  {best_feature} with p-value {best_pval:.4f}')
        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            if verbose:
                print(f'Drop {worst_feature} with p-value {worst_pval:.4f}')
        if not changed:
           break
    return included


In [25]:
X = df.drop(columns=['Target_Y'])
y = df['Target_Y']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X = pd.DataFrame(X_scaled, columns=X.columns)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=69)

In [28]:
stepwise_selection(X, y)

Add  X5 with p-value 0.0000
Add  X28 with p-value 0.0000
Add  X23 with p-value 0.0000
Add  X27_windows with p-value 0.0000
Add  X22 with p-value 0.0000
Add  X19 with p-value 0.0000
Add  X2 with p-value 0.0000
Add  X1 with p-value 0.0000
Add  X8_AC with p-value 0.0000
Add  X18_BC with p-value 0.0000
Add  X18_BB with p-value 0.0000
Add  X18_BE with p-value 0.0000
Add  X18_BD with p-value 0.0000
Add  X29 with p-value 0.0000
Add  X17 with p-value 0.0000
Add  X3 with p-value 0.0000
Add  X16 with p-value 0.0000
Add  X15_CB with p-value 0.0000
Add  X21 with p-value 0.0000
Add  X24 with p-value 0.0000
Add  X14 with p-value 0.0000
Add  X7 with p-value 0.0000
Add  X9 with p-value 0.0000
Add  X27_macintosh with p-value 0.0000
Add  X15_CC with p-value 0.0000
Add  X18_BF with p-value 0.0000
Add  X27_other with p-value 0.0000
Add  X31 with p-value 0.0000
Add  X25_TELEAPP with p-value 0.0000
Add  X15_CD with p-value 0.0000
Add  X15_CF with p-value 0.0000
Add  X20 with p-value 0.0000
Add  X6 with p-va

['X5',
 'X28',
 'X23',
 'X27_windows',
 'X22',
 'X19',
 'X2',
 'X1',
 'X8_AC',
 'X18_BC',
 'X18_BB',
 'X18_BE',
 'X18_BD',
 'X29',
 'X17',
 'X3',
 'X16',
 'X15_CB',
 'X21',
 'X24',
 'X14',
 'X7',
 'X9',
 'X27_macintosh',
 'X15_CC',
 'X18_BF',
 'X27_other',
 'X31',
 'X25_TELEAPP',
 'X15_CD',
 'X15_CF',
 'X20',
 'X6',
 'X4',
 'X13',
 'X15_CE',
 'X18_BG',
 'X27_x11']

In [29]:
chosen_columns = ['X5',
 'X28',
 'X23',
 'X27_windows',
 'X22',
 'X19',
 'X2',
 'X1',
 'X8_AC',
 'X18_BC',
 'X18_BB',
 'X18_BE',
 'X18_BD',
 'X29',
 'X17',
 'X3',
 'X16',
 'X15_CB',
 'X21',
 'X24',
 'X14',
 'X7',
 'X9',
 'X27_macintosh',
 'X15_CC',
 'X18_BF',
 'X27_other',
 'X31',
 'X25_TELEAPP',
 'X15_CD',
 'X15_CF',
 'X20',
 'X6',
 'X4',
 'X13',
 'X15_CE',
 'X18_BG',
 'X27_x11']
print(len(chosen_columns))

38


In [30]:
X = X[chosen_columns]

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=69)

In [36]:
model = Sequential()
model.add(Dense(19, activation='tanh', input_shape=(38,)))
model.add(Dense(1, activation='sigmoid', input_shape=(19,)))

model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy', 'precision', 'recall'])
model.fit(X_train, y_train, epochs=100, batch_size=64, verbose=1)

c:\Users\trush\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
10938/10938 ━━━━━━━━━━━━━━━━━━━━ 24s 2ms/step - accuracy: 0.9672 - loss: 0.1070 - precision: 0.0255 - recall: 0.0321
Epoch 2/100
10938/10938 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - accuracy: 0.9890 - loss: 0.0466 - precision: 0.5765 - recall: 0.0113
Epoch 3/100
10938/10938 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - accuracy: 0.9891 - loss: 0.0462 - precision: 0.6032 - recall: 0.0111
Epoch 4/100
10938/10938 ━━━━━━━━━━━━━━━━━━━━ 22s 2ms/step - accuracy: 0.9891 - loss: 0.0461 - precision: 0.6612 - recall: 0.0119
Epoch 5/100
10938/10938 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - accuracy: 0.9891 - loss: 0.0453 - precision: 0.5007 - recall: 0.0111
Epoch 6/100
10938/10938 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - accuracy: 0.9890 - loss: 0.0457 - precision: 0.5886 - recall: 0.0136
Epoch 7/100
10938/10938 ━━━━━━━━━━━━━━━━━━━━ 16s 1ms/step - accuracy: 0.9890 - loss: 0.0458 - precision: 0.5283 - recall: 0.0122
Epoch 8/100
10938/10938 ━━━━━━━━━━━━━━━━━━━━ 16s 1ms/step - accuracy: 0.9893 - loss: 0.0446 - pre

KeyboardInterrupt: 